# Fuel Economy Data &mdash; Part 4

### This notebook contains the webscraping process I took to grab images for all unique vehicles. I also created interesting summary/aggregate columns to display in my web app.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time

bing_url= 'https://www.bing.com/images/search?q='
bing_url_suffix='&first=1&tsc=ImageBasicHover'

fuelgov_url = 'https://www.fueleconomy.gov/feg/Find.do?action=sbs&id='

In [2]:
# load cleaned dataset from Part 1 of the homework
df = pd.read_csv('./data/fuel_eco_clean.csv')

## Purpose and Background

I want to enhance my streamlit presentation by allowing users to get interesting data on various car models and also see images of the vehicles, if available. I had a few false starts in this process. I first tried to scrape Google image results, but I was quickly throttled into oblivion. Bing allowed me to send more requests per hour, but the general quality of the data was lower.

Finally, I discovered that fueleconomy.gov has images for many cars. As an added bonus, finding pages for vehicles was easy because pages are identified by vehicle ID. So I will first generate a list of unique vehicles (including vehicle ID):

#### Creating a list of unique vehicles to webscrape for images with:

In [266]:
m = df.groupby(['Year', 'Make', 'Model'])
m = m.first()
models_unique = m.reset_index()
models_unique = models_unique[['Year', 'Make', 'Model', 'Vehicle ID']]

In [267]:
models_unique

,Year,Make,Model,Vehicle ID
0,1984,AM General,DJ Po Vehicle 2WD,27550
1,1984,AM General,FJ8c Post Office,27549
2,1984,Alfa Romeo,GT V6 2.5,26587
3,1984,Alfa Romeo,Spider Veloce 2000,26561
4,1984,American Motors Corporation,Eagle 4WD,27593
...,...,...,...,...
17205,2017,Volvo,XC90 AWD,37547
17206,2017,Volvo,XC90 AWD PHEV,37703
17207,2017,Volvo,XC90 FWD,37687
17208,2017,smart,fortwo cabriolet,37873


#### Testing fueleconomy.gov page results to find images

In [208]:
page = requests.get(fuelgov_url+str(38138))

In [209]:
soup = BeautifulSoup(page.content, 'html.parser')

In [212]:
soup.find_all('img', class_="thumbnail")[1]['src']




'/feg/photos/2017_Aston_Martin_Vanquish_Zegato.jpg'

I tested several pages, and it looks like the second image with the thumbnail class is consistently the target.

In [213]:
img_dict = dict()

I will initialize a dictionary **outside** of my function because I want the program to be able to recover and start where it left off if the connection gets reset. I had several hundred scraping results get lost because the get_img_urls function (below) intially didn't return anything until the very end. 
<br>
<br>
Print statements allow me to monitor the progress of the function. I also added logic to this function to check the length of img_dict and resume grabbing images at that point:

In [265]:
def get_img_urls(dataset, d, url):
    print(f"Starting at {len(d)}...")
    for i in range(len(d), len(dataset.index)):
        year = str(dataset['Year'][i])
        make = str(dataset['Make'][i])
        model = str(dataset['Model'][i])
        v_id = str(dataset['Vehicle ID'][i])
        page = requests.get(url+v_id)
        soup = BeautifulSoup(page.content, 'html.parser')
        print(year+' '+make+' '+model)
        try:
            img = soup.find_all('img', class_="thumbnail")[1]['src']
            d[i] = 'https://fueleconomy.gov' + img
            print(i, "found and added.", len(d))
        except:
            #placeholder image in case no google image is found for the car model
            d[i]='https://www.fueleconomy.gov/feg/photos/nophoto.gif'
            print(i, "not found. Added generic instead", len(d))
        # time.sleep(1)
    return d

In [222]:
# Code used to grab webscraper results. This latest output starts at 10 because I briefly ran the
# program just to test that it worked as expected.

# imgs = get_img_urls(models_unique, img_dict, fuelgov_url)

Starting at 10...
1984 Audi 5000S
10 found and added. 11
1984 Audi 5000S Wagon
11 found and added. 12
1984 Audi Coupe GT
12 found and added. 13
1984 Audi Quattro
13 found and added. 14
1984 Avanti Motor Corporation Avanti II
14 found and added. 15
1984 BMW 3 Series
15 found and added. 16
1984 BMW 5 Series
16 found and added. 17
1984 BMW 6 Series
17 found and added. 18
1984 BMW 7 Series
18 found and added. 19
1984 Bertone X1/9
19 found and added. 20
1984 Bill Dovell Motor Car Company Dovell 230CE
20 found and added. 21
1984 Bill Dovell Motor Car Company Dovell 230E
21 found and added. 22
1984 Bitter Gmbh and Co. Kg SC
22 found and added. 23
1984 Buick Century
23 found and added. 24
1984 Buick Century Estate Wagon
24 found and added. 25
1984 Buick Electra Estate Wagon
25 found and added. 26
1984 Buick Electra/Park Avenue
26 found and added. 27
1984 Buick LeSabre
27 found and added. 28
1984 Buick Regal
28 found and added. 29
1984 Buick Riviera
29 found and added. 30
1984 Buick Riviera Con

1984 Lamborghini Countach Lp500s
171 found and added. 172
1984 Lincoln Continental
172 found and added. 173
1984 Lincoln Mark VII
173 found and added. 174
1984 Lincoln Town Car
174 found and added. 175
1984 Lotus Esprit
175 found and added. 176
1984 Maserati Biturbo
176 found and added. 177
1984 Maserati Quattroporte
177 found and added. 178
1984 Mazda 626
178 found and added. 179
1984 Mazda B2000/B2200 Pickup 2WD
179 found and added. 180
1984 Mazda GLC
180 found and added. 181
1984 Mazda GLC Wagon
181 found and added. 182
1984 Mazda RX-7
182 found and added. 183
1984 Mercedes-Benz 190 D 2.2/190 E 2.3
183 found and added. 184
1984 Mercedes-Benz 300D/300CD
184 found and added. 185
1984 Mercedes-Benz 300SD/380SE
185 found and added. 186
1984 Mercedes-Benz 300TD
186 found and added. 187
1984 Mercedes-Benz 380SL
187 found and added. 188
1984 Mercedes-Benz 500SEC
188 found and added. 189
1984 Mercedes-Benz 500SEL
189 found and added. 190
1984 Mercury Capri
190 found and added. 191
1984 Merc

1985 Buick Skyhawk
334 found and added. 335
1985 Buick Skyhawk Wagon
335 found and added. 336
1985 Buick Skylark
336 found and added. 337
1985 Buick Somerset Regal
337 found and added. 338
1985 Cadillac Cimarron
338 found and added. 339
1985 Cadillac Eldorado
339 found and added. 340
1985 Cadillac Eldorado Convertible
340 found and added. 341
1985 Cadillac Fleetwood Brougham (RWD)
341 found and added. 342
1985 Cadillac Fleetwood/DeVille (FWD)
342 found and added. 343
1985 Cadillac Limousine
343 found and added. 344
1985 Cadillac Seville
344 found and added. 345
1985 Chevrolet Astro 2WD (cargo)
345 found and added. 346
1985 Chevrolet Astro 2WD (passenger)
346 found and added. 347
1985 Chevrolet C10 Pickup 2WD
347 found and added. 348
1985 Chevrolet C20 Pickup 2WD
348 found and added. 349
1985 Chevrolet Camaro
349 found and added. 350
1985 Chevrolet Caprice Wagon
350 found and added. 351
1985 Chevrolet Cavalier
351 found and added. 352
1985 Chevrolet Cavalier Convertible
352 found and ad

1985 Mercedes-Benz 300TD
496 found and added. 497
1985 Mercedes-Benz 380SE
497 found and added. 498
1985 Mercedes-Benz 380SL
498 found and added. 499
1985 Mercedes-Benz 500SE
499 found and added. 500
1985 Mercedes-Benz 500SEC
500 found and added. 501
1985 Mercedes-Benz 500SEL
501 found and added. 502
1985 Mercedes-Benz 500SL
502 found and added. 503
1985 Mercury Capri
503 found and added. 504
1985 Mercury Cougar
504 found and added. 505
1985 Mercury Grand Marquis
505 found and added. 506
1985 Mercury Grand Marquis Wagon
506 found and added. 507
1985 Mercury Lynx
507 found and added. 508
1985 Mercury Lynx Wagon
508 found and added. 509
1985 Mercury Marquis
509 found and added. 510
1985 Mercury Marquis Wagon
510 found and added. 511
1985 Mercury Topaz
511 found and added. 512
1985 Merkur XR4Ti
512 found and added. 513
1985 Mitsubishi Cordia
513 found and added. 514
1985 Mitsubishi Galant
514 found and added. 515
1985 Mitsubishi Mirage
515 found and added. 516
1985 Mitsubishi Montero 4WD


1986 Chevrolet Astro 2WD (cargo)
662 found and added. 663
1986 Chevrolet Astro 2WD (passenger)
663 found and added. 664
1986 Chevrolet C10 Pickup 2WD
664 found and added. 665
1986 Chevrolet C20 Pickup 2WD
665 found and added. 666
1986 Chevrolet Camaro
666 found and added. 667
1986 Chevrolet Caprice
667 found and added. 668
1986 Chevrolet Caprice Wagon
668 found and added. 669
1986 Chevrolet Cavalier
669 found and added. 670
1986 Chevrolet Cavalier Convertible
670 found and added. 671
1986 Chevrolet Cavalier Wagon
671 found and added. 672
1986 Chevrolet Celebrity
672 found and added. 673
1986 Chevrolet Celebrity Wagon
673 found and added. 674
1986 Chevrolet Chevette CS
674 found and added. 675
1986 Chevrolet Corvette
675 found and added. 676
1986 Chevrolet Corvette Convertible
676 found and added. 677
1986 Chevrolet El Camino Pickup 2WD
677 found and added. 678
1986 Chevrolet G10/20 Sport Van 2WD
678 found and added. 679
1986 Chevrolet G10/20 Van 2WD
679 found and added. 680
1986 Chevro

...

2017 Buick Verano
16549 found and added. 16550
2017 Cadillac ATS
16550 found and added. 16551
2017 Cadillac ATS AWD
16551 found and added. 16552
2017 Cadillac ATS-V
16552 found and added. 16553
2017 Cadillac CT6
16553 found and added. 16554
2017 Cadillac CT6 AWD
16554 found and added. 16555
2017 Cadillac CTS
16555 found and added. 16556
2017 Cadillac CTS AWD
16556 found and added. 16557
2017 Cadillac CTS-V
16557 found and added. 16558
2017 Cadillac Escalade 2WD
16558 found and added. 16559
2017 Cadillac Escalade 4WD
16559 found and added. 16560
2017 Cadillac XT5
16560 found and added. 16561
2017 Cadillac XT5 AWD
16561 found and added. 16562
2017 Cadillac XTS
16562 found and added. 16563
2017 Cadillac XTS AWD
16563 found and added. 16564
2017 Cadillac XTS Hearse
16564 found and added. 16565
2017 Cadillac XTS LIMO
16565 found and added. 16566
2017 Chevrolet Bolt
16566 found and added. 16567
2017 Chevrolet Camaro
16567 found and added. 16568
2017 Chevrolet City Express Cargo Van
16568 fou

2017 GMC Acadia FWD
16697 found and added. 16698
2017 GMC Acadia Limited AWD
16698 found and added. 16699
2017 GMC Acadia Limited FWD
16699 found and added. 16700
2017 GMC Canyon 2WD
16700 found and added. 16701
2017 GMC Canyon 4WD
16701 found and added. 16702
2017 GMC Canyon Cab Chassis 2WD
16702 found and added. 16703
2017 GMC Savana 2500 2WD (Passenger)
16703 found and added. 16704
2017 GMC Savana 3500 2WD (Passenger)
16704 found and added. 16705
2017 GMC Sierra 15 Hybrid 2WD
16705 found and added. 16706
2017 GMC Sierra 15 Hybrid 4WD
16706 found and added. 16707
2017 GMC Sierra C15 2WD
16707 found and added. 16708
2017 GMC Sierra C15 Cab Chassis 2WD
16708 found and added. 16709
2017 GMC Sierra K15 4WD
16709 found and added. 16710
2017 GMC Sierra K15 Cab Chassis 4WD
16710 found and added. 16711
2017 GMC Terrain AWD
16711 found and added. 16712
2017 GMC Terrain FWD
16712 found and added. 16713
2017 GMC Yukon C1500 2WD
16713 found and added. 16714
2017 GMC Yukon C1500 XL 2WD
16714 foun

2017 Lamborghini Aventador Roadster
16851 found and added. 16852
2017 Lamborghini Aventador Roadster LP740-4
16852 found and added. 16853
2017 Lamborghini Huracan
16853 found and added. 16854
2017 Lamborghini Huracan 2WD
16854 found and added. 16855
2017 Lamborghini Huracan Spyder
16855 found and added. 16856
2017 Lamborghini Huracan Spyder 2WD
16856 found and added. 16857
2017 Land Rover Discovery Sport
16857 found and added. 16858
2017 Land Rover Range Rover
16858 found and added. 16859
2017 Land Rover Range Rover Evoque
16859 found and added. 16860
2017 Land Rover Range Rover Evoque Convertible
16860 found and added. 16861
2017 Land Rover Range Rover LWB
16861 found and added. 16862
2017 Land Rover Range Rover Sport
16862 found and added. 16863
2017 Lexus CT 200h
16863 found and added. 16864
2017 Lexus ES 300h
16864 found and added. 16865
2017 Lexus ES 350
16865 found and added. 16866
2017 Lexus GS 200t
16866 found and added. 16867
2017 Lexus GS 200t F Sport
16867 found and added. 1

2017 Mercedes-Benz E550 Convertible
16995 found and added. 16996
2017 Mercedes-Benz E550 Coupe
16996 found and added. 16997
2017 Mercedes-Benz G550
16997 found and added. 16998
2017 Mercedes-Benz GLA250
16998 found and added. 16999
2017 Mercedes-Benz GLA250 4matic
16999 found and added. 17000
2017 Mercedes-Benz GLC300
17000 found and added. 17001
2017 Mercedes-Benz GLC300 4matic
17001 found and added. 17002
2017 Mercedes-Benz GLC300 4matic Coupe
17002 found and added. 17003
2017 Mercedes-Benz GLE350
17003 found and added. 17004
2017 Mercedes-Benz GLE350 4matic
17004 found and added. 17005
2017 Mercedes-Benz GLE400 4matic
17005 found and added. 17006
2017 Mercedes-Benz GLE550e 4matic
17006 found and added. 17007
2017 Mercedes-Benz GLS450 4matic
17007 found and added. 17008
2017 Mercedes-Benz GLS550 4matic
17008 found and added. 17009
2017 Mercedes-Benz Maybach S550 4matic
17009 found and added. 17010
2017 Mercedes-Benz Maybach S600
17010 found and added. 17011
2017 Mercedes-Benz Metris 

2017 Toyota Corolla iM
17141 found and added. 17142
2017 Toyota Highlander
17142 found and added. 17143
2017 Toyota Highlander AWD
17143 found and added. 17144
2017 Toyota Highlander AWD LE
17144 found and added. 17145
2017 Toyota Highlander Hybrid 4WD
17145 found and added. 17146
2017 Toyota Highlander Hybrid 4WD LE Plus
17146 found and added. 17147
2017 Toyota Highlander LE/XLE/SE/LTD
17147 found and added. 17148
2017 Toyota Land Cruiser Wagon 4WD
17148 found and added. 17149
2017 Toyota Prius
17149 found and added. 17150
2017 Toyota Prius Eco
17150 found and added. 17151
2017 Toyota Prius Prime
17151 found and added. 17152
2017 Toyota Prius c
17152 found and added. 17153
2017 Toyota Prius v
17153 found and added. 17154
2017 Toyota RAV4
17154 found and added. 17155
2017 Toyota RAV4 AWD
17155 found and added. 17156
2017 Toyota RAV4 Hybrid AWD
17156 found and added. 17157
2017 Toyota RAV4 LE/XLE
17157 found and added. 17158
2017 Toyota RAV4 Limited AWD/SE AWD
17158 found and added. 171

In [249]:
imgs_df = pd.DataFrame.from_dict(img_dict, orient='index', columns=['Image Url'])

In [250]:
models_unique['Image Url'] = imgs_df

In [264]:
models_unique.head()

,Year,Make,Model,Vehicle ID,Image Url
0,1984,AM General,DJ Po Vehicle 2WD,27550,https://fueleconomy.gov/feg/photos/nophoto.gif
1,1984,AM General,FJ8c Post Office,27549,https://fueleconomy.gov/feg/photos/nophoto.gif
2,1984,Alfa Romeo,GT V6 2.5,26587,https://fueleconomy.gov/feg/photos/1984_Alfa_R...
3,1984,Alfa Romeo,Spider Veloce 2000,26561,https://fueleconomy.gov/feg/photos/nophoto.gif
4,1984,American Motors Corporation,Eagle 4WD,27593,https://fueleconomy.gov/feg/photos/1984_AMC_Ea...


I checked that the img_dict had the same length as the models_unique dataframe. I also randomly sampled different indices and checked that the url sent me to the correct vehicle. Looks like everything works! The next step is outputting my data to a csv file that I can then load into my streamlit app.

In [236]:
#models_unique.to_csv('./data/fuelgov_img_urls.csv', index = False, header=True, encoding='utf-8-sig')

In addition to displaying the vehicle picture and information, it would be interesting to show how other vehicles made that same year compare to its MPG. I would like to see statistics on vehicles of the same class, same make, same fuel type that were made the same year as the target vehicle.

In [25]:
class_mean = df.groupby(['Class', 'Year'])['Combined MPG (FT1)'].mean()
df = df.merge(class_mean, on=['Class', 'Year'])

df = df.rename(columns={'Combined MPG (FT1)_x':'Combined MPG (FT1)', 
                  'Combined MPG (FT1)_y':'MPG (Class Average)'})


df['MPG - Class Ave. Difference'] = df['Combined MPG (FT1)'] - df['MPG (Class Average)']
df.head()


,Vehicle ID,Year,Make,Model,Class,Drive,Transmission,Transmission Descriptor,Engine Index,Engine Descriptor,...,Composite Combined MPG,Range (FT1),City Range (FT1),Highway Range (FT1),City Range (FT2),Highway Range (FT2),Range (FT2) Clean,Manufacturer Code Clean,MPG (Class Average),MPG - Class Ave. Difference
0,26587,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,Unknown,Manual 5-Speed,Unknown,9001,(FFS),...,0,0,0.0,0.0,0.0,0.0,0,CRX,20.173913,-0.173913
1,27705,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,Unknown,Manual 5-Speed,Unknown,9005,(FFS) CA model,...,0,0,0.0,0.0,0.0,0.0,0,CRX,20.173913,-0.173913
2,26588,1984,Avanti Motor Corporation,Avanti II,Minicompact Cars,Unknown,Automatic 4-Speed,Unknown,10010,(FFS),...,0,0,0.0,0.0,0.0,0.0,0,Unknown,20.173913,-4.173913
3,27706,1984,Avanti Motor Corporation,Avanti II,Minicompact Cars,Unknown,Automatic 4-Speed,Unknown,10015,(FFS) CA model,...,0,0,0.0,0.0,0.0,0.0,0,Unknown,20.173913,-3.173913
4,26589,1984,Maserati,Biturbo,Minicompact Cars,Unknown,Manual 5-Speed,Unknown,36001,(GUZZLER),...,0,0,0.0,0.0,0.0,0.0,0,MAX,20.173913,-6.173913


In [26]:
make_mean = df.groupby(['Make', 'Year'])['Combined MPG (FT1)'].mean()
df = df.merge(make_mean, on=['Make', 'Year'])

df = df.rename(columns={'Combined MPG (FT1)_x':'Combined MPG (FT1)', 
                  'Combined MPG (FT1)_y':'MPG (Make Average)'})


df['MPG - Make Ave. Difference'] = df['Combined MPG (FT1)'] - df['MPG (Make Average)']
df.head()


,Vehicle ID,Year,Make,Model,Class,Drive,Transmission,Transmission Descriptor,Engine Index,Engine Descriptor,...,City Range (FT1),Highway Range (FT1),City Range (FT2),Highway Range (FT2),Range (FT2) Clean,Manufacturer Code Clean,MPG (Class Average),MPG - Class Ave. Difference,MPG (Make Average),MPG - Make Ave. Difference
0,26587,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,Unknown,Manual 5-Speed,Unknown,9001,(FFS),...,0.0,0.0,0.0,0.0,0,CRX,20.173913,-0.173913,20.5,-0.5
1,27705,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,Unknown,Manual 5-Speed,Unknown,9005,(FFS) CA model,...,0.0,0.0,0.0,0.0,0,CRX,20.173913,-0.173913,20.5,-0.5
2,26561,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,Unknown,Manual 5-Speed,Unknown,9002,(FFS),...,0.0,0.0,0.0,0.0,0,CRX,20.540000,0.460000,20.5,0.5
3,27681,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,Unknown,Manual 5-Speed,Unknown,9006,(FFS) CA model,...,0.0,0.0,0.0,0.0,0,CRX,20.540000,0.460000,20.5,0.5
4,26588,1984,Avanti Motor Corporation,Avanti II,Minicompact Cars,Unknown,Automatic 4-Speed,Unknown,10010,(FFS),...,0.0,0.0,0.0,0.0,0,Unknown,20.173913,-4.173913,16.5,-0.5


In [27]:
fueltype_mean = df.groupby(['Fuel Type', 'Year'])['Combined MPG (FT1)'].mean()
df = df.merge(fueltype_mean, on=['Fuel Type', 'Year'])

df = df.rename(columns={'Combined MPG (FT1)_x':'Combined MPG (FT1)', 
                  'Combined MPG (FT1)_y':'MPG (Fuel Type Average)'})


df['MPG - Fuel Type Ave. Difference'] = df['Combined MPG (FT1)'] - df['MPG (Fuel Type Average)']
df.head()

,Vehicle ID,Year,Make,Model,Class,Drive,Transmission,Transmission Descriptor,Engine Index,Engine Descriptor,...,City Range (FT2),Highway Range (FT2),Range (FT2) Clean,Manufacturer Code Clean,MPG (Class Average),MPG - Class Ave. Difference,MPG (Make Average),MPG - Make Ave. Difference,MPG (Fuel Type Average),MPG - Fuel Type Ave. Difference
0,26587,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,Unknown,Manual 5-Speed,Unknown,9001,(FFS),...,0.0,0.0,0,CRX,20.173913,-0.173913,20.5,-0.5,19.121622,0.878378
1,27705,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,Unknown,Manual 5-Speed,Unknown,9005,(FFS) CA model,...,0.0,0.0,0,CRX,20.173913,-0.173913,20.5,-0.5,19.121622,0.878378
2,26561,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,Unknown,Manual 5-Speed,Unknown,9002,(FFS),...,0.0,0.0,0,CRX,20.540000,0.460000,20.5,0.5,19.121622,1.878378
3,27681,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,Unknown,Manual 5-Speed,Unknown,9006,(FFS) CA model,...,0.0,0.0,0,CRX,20.540000,0.460000,20.5,0.5,19.121622,1.878378
4,26588,1984,Avanti Motor Corporation,Avanti II,Minicompact Cars,Unknown,Automatic 4-Speed,Unknown,10010,(FFS),...,0.0,0.0,0,Unknown,20.173913,-4.173913,16.5,-0.5,19.121622,-3.121622


In [28]:
year_mean = df.groupby(['Year'])['Combined MPG (FT1)'].mean()
df = df.merge(year_mean, on=['Year'])

df = df.rename(columns={'Combined MPG (FT1)_x':'Combined MPG (FT1)', 
                  'Combined MPG (FT1)_y':'MPG (Year Average)'})


df['MPG - Year Ave. Difference'] = df['Combined MPG (FT1)'] - df['MPG (Year Average)']
df.head()

,Vehicle ID,Year,Make,Model,Class,Drive,Transmission,Transmission Descriptor,Engine Index,Engine Descriptor,...,Range (FT2) Clean,Manufacturer Code Clean,MPG (Class Average),MPG - Class Ave. Difference,MPG (Make Average),MPG - Make Ave. Difference,MPG (Fuel Type Average),MPG - Fuel Type Ave. Difference,MPG (Year Average),MPG - Year Ave. Difference
0,26587,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,Unknown,Manual 5-Speed,Unknown,9001,(FFS),...,0,CRX,20.173913,-0.173913,20.5,-0.5,19.121622,0.878378,19.881874,0.118126
1,27705,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,Unknown,Manual 5-Speed,Unknown,9005,(FFS) CA model,...,0,CRX,20.173913,-0.173913,20.5,-0.5,19.121622,0.878378,19.881874,0.118126
2,26561,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,Unknown,Manual 5-Speed,Unknown,9002,(FFS),...,0,CRX,20.540000,0.460000,20.5,0.5,19.121622,1.878378,19.881874,1.118126
3,27681,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,Unknown,Manual 5-Speed,Unknown,9006,(FFS) CA model,...,0,CRX,20.540000,0.460000,20.5,0.5,19.121622,1.878378,19.881874,1.118126
4,26588,1984,Avanti Motor Corporation,Avanti II,Minicompact Cars,Unknown,Automatic 4-Speed,Unknown,10010,(FFS),...,0,Unknown,20.173913,-4.173913,16.5,-0.5,19.121622,-3.121622,19.881874,-3.881874


In [29]:
# df.to_csv('./data/fuel_eco_clean_averages.csv', index = False, header=True, encoding='utf-8-sig')